# DSCI 525: Web and Cloud Computing

## Milestone 1: Tackling Big Data on Computer

### Group 13
Authors: Ivy Zhang, Mike Lynch, Selma Duric, William Xu

## Table of contents

- [Download the data](#1)
- [Combining data CSVs](#2)
- [Load the combined CSV to memory and perform a simple EDA](#3)
- [Perform a simple EDA in R](#4)
- [Reflection](#5)

### Imports

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
import pyarrow.feather as feather
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Download the data <a name="1"></a>

1. Download the data from figshare to local computer using the figshare API.
2. Extract the zip file programmatically.

In [3]:
# Attribution: DSCI 525 lecture notebook
# Necessary metadata
article_id = 14096681  # unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]    

In [5]:
%%time
files_to_dl = ["data.zip"]  
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.88 s, sys: 2.93 s, total: 6.81 s
Wall time: 1min 10s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 10.8 s, sys: 1.49 s, total: 12.3 s
Wall time: 12.3 s


## Combining data CSVs <a name="2"></a>

1. Use one of the following options to combine data CSVs into a single CSV (Pandas, Dask). **We used the option of Pandas**.
2. When combining the csv files, we added extra column called "model" that identifies the model (we get this column populated from the file name eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON)
3. Compare run times and memory usages of these options on different machines within the team, and summarize observations.

In [7]:
%%time
%memit
# Shows time that regular python takes to merge file
# Join all data together
## here we are using a normal python way of merging the data 
# use_cols = ["time", "lat_min", "lat_max", "lon_min","lon_max","rain (mm/day)"]
files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )
df.to_csv("figsharerainfall/combined_data.csv")

peak memory: 168.59 MiB, increment: 0.12 MiB
CPU times: user 3min 29s, sys: 5.09 s, total: 3min 34s
Wall time: 3min 34s


In [8]:
feather.write_feather(df, "figsharerainfall/combined_data.feather")

In [9]:
%%sh
du -sh figsharerainfall/combined_data.csv

6.6G	figsharerainfall/combined_data.csv


In [10]:
%%time
df = pd.read_csv("figsharerainfall/combined_data.csv")

CPU times: user 33.3 s, sys: 3.11 s, total: 36.5 s
Wall time: 37.1 s


In [11]:
print(df.shape)

(62513863, 7)


In [12]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,figsharerainfall/MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,figsharerainfall/MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,figsharerainfall/MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,figsharerainfall/MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,figsharerainfall/MPI-ESM-1-2-HAM


**Summary of run times and memory usages:**

***William***
- Combining files: 
    - peak memory: 95.41 MiB, increment: 0.26 MiB
    - CPU times: user 7min 28s, sys: 31 s, total: 7min 59s
    - Wall time: 9min 17s
- Reading the combined file:
    - Wall time: 1min 51s

***Mike***
- Combining files: 
    - peak memory: 168.59 MiB, increment: 0.12 MiB
    - CPU times: user 3min 29s, sys: 5.09 s, total: 3min 34s
    - Wall time: 3min 34s
- Reading the combined file:
    - Wall time: 37.1 s

***Selma***
- Combining files: 
    - peak memory: 150.54 MiB, increment: 0.23 MiB
    - CPU times: user 6min 46s, sys: 23.1 s, total: 7min 9s
    - Wall time: 7min 29s
- Reading the combined file:
    - Wall time: 1min 19s
    


## Load the combined CSV to memory and perform a simple EDA <a name="3"></a>

### Establish a baseline for memory usage

In [3]:
df = pd.read_csv("figsharerainfall/combined_data.csv", parse_dates=True, index_col='time')
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.286930e-24,figsharerainfall/BCC-ESM1
1889-01-02 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.279780e-24,figsharerainfall/BCC-ESM1
1889-01-03 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.499248e-17,figsharerainfall/BCC-ESM1
1889-01-04 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.013407e-01,figsharerainfall/BCC-ESM1
1889-01-05 12:00:00,-36.277805,-33.487232,142.03125,144.84375,4.578585e-01,figsharerainfall/BCC-ESM1


In [4]:
df.dtypes

lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [5]:
%%time
%memit
df.describe()

peak memory: 4578.73 MiB, increment: 0.05 MiB
CPU times: user 5.98 s, sys: 1.19 s, total: 7.17 s
Wall time: 7.34 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901827e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588275e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


Baseline memory and time data:
- peak memory: 4578.73 MiB, increment: 0.05 MiB
- CPU times: user 5.98 s, sys: 1.19 s, total: 7.17 s
- Wall time: 7.34 s

### Effects of changing dtypes on memory usage

In [6]:
colum_dtypes = {'lat_min': np.float32, 'lat_max': np.float32, 'lon_min': np.float32, 'lon_max': np.float32, 'rain (mm/day)': np.float32, 'model': str}
df = pd.read_csv("figsharerainfall/combined_data.csv",parse_dates=True, index_col='time', dtype=colum_dtypes)
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.286930e-24,figsharerainfall/BCC-ESM1
1889-01-02 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.279780e-24,figsharerainfall/BCC-ESM1
1889-01-03 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.499248e-17,figsharerainfall/BCC-ESM1
1889-01-04 12:00:00,-36.277805,-33.487232,142.03125,144.84375,1.013407e-01,figsharerainfall/BCC-ESM1
1889-01-05 12:00:00,-36.277805,-33.487232,142.03125,144.84375,4.578585e-01,figsharerainfall/BCC-ESM1


In [7]:
df.dtypes

lat_min          float32
lat_max          float32
lon_min          float32
lon_max          float32
rain (mm/day)    float32
model             object
dtype: object

In [8]:
%%time
%memit
df.describe()

peak memory: 6316.88 MiB, increment: 0.18 MiB
CPU times: user 4.81 s, sys: 761 ms, total: 5.58 s
Wall time: 5.73 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.929456e+07
mean,-3.310503e+01,-3.197712e+01,1.469070e+02,1.482141e+02,1.901825e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588274e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


Time and memory data when using different dtypes:
- peak memory: 6316.88 MiB, increment: 0.18 MiB
- CPU times: user 4.81 s, sys: 761 ms, total: 5.58 s
- Wall time: 5.73 s

### Effects of loading a smaller subset of columns on memory usage

In [9]:
df = pd.read_csv("figsharerainfall/combined_data.csv",parse_dates=True, index_col='time', usecols=['time', 'lat_min', 'rain (mm/day)'])
df.head()

In [10]:
df.dtypes

lat_min          float64
rain (mm/day)    float64
dtype: object

In [11]:
%%time
%memit
df.describe()

peak memory: 7748.36 MiB, increment: 0.02 MiB
CPU times: user 2.74 s, sys: 1.08 s, total: 3.82 s
Wall time: 4 s


,lat_min,rain (mm/day)
count,5.924854e+07,5.929456e+07
mean,-3.310482e+01,1.901827e+00
std,1.963549e+00,5.588275e+00
min,-3.646739e+01,-3.807373e-12
25%,-3.486911e+01,3.876672e-06
50%,-3.300000e+01,6.161705e-02
75%,-3.140170e+01,1.021314e+00
max,-2.990000e+01,4.329395e+02


Time and memory data when using column subset:
- peak memory: 7748.36 MiB, increment: 0.02 MiB
- CPU times: user 2.74 s, sys: 1.08 s, total: 3.82 s
- Wall time: 4 s

### Summary

#### Using float32 vs. baseline float64 dtype to perform a simple EDA:
- When using the pandas describe function to perform a simple EDA, we found that the memory usage actually increased when using float32 dtype for the numerical columns instead of float64. 
- The time taken to perform the EDA also decreased compared to the baseline. 

#### Using a reduced number of columns compared to the baseline to perform a simple EDA:
- Interestingly, the memory usage actually increased substantially compared to the baseline when performing a simple EDA on the reduced dataset.
- The time taken to perform the EDA also decreased compared to the baseline.

## Perform a simple EDA in R <a name="4"></a>

## Reflection <a name="5"></a>